# Example 1: Measuring shape, size and colour of isopods  

Little nugget of information: this was the original computer vision problem that phenopype was intended to solve (it's predecessor ["iso_cv"](https://github.com/mluerig/iso_cv) was not much more than a script-collection).

In this example I am demonstrating all three phenopype workflows ([prototyping](#Prototyping), [low thoughput](#Low-throughput) and [high throughput](#High-throughput)). For a project, you would probably only use either the high or low throughput approach, but the protoypting approach for this example decomposes the all the required steps for a classic computer vision workflow. 

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_assets/ex1_before.jpg)
    
**Input** - Freshwater isopod, alive, photographed on a white resin-tray from a camera stand. 
</div>
<div class="col-md-6">

![After](_assets/ex1_after.jpg)
    
**Results** - Isopod shape, size and colour are extracted (and size referenced using the reference card) 
</div>
</div>

## Prototyping

### Loading the image

First we import the image from the a filepath using `load_image`. With the flag `df=True` we can extract some image-meta information that we want to be represented in the results files later (i.e. image name and its dimensions). After loading it, we can have a quick look at it with the `show_image` function - you can close it again with Enter or Esc. 

In [1]:
import phenopype as pp

In [2]:
filepath = r"images/isopods.jpg"
image, img_data = pp.load_image(filepath, df=True)
pp.show_image(image)

In [3]:
img_data ## size ratio refers whether this image has been resized using the resize argument of `load_image`

,filename,width,height,size_ratio_original
0,isopods.jpg,2100,1400,1


### Drawing a mask

The original image has a lot of noise, e.g. the non-white area around the tray, water reflections, the label, the reference card, and the little fecal pellets that lie around on the tray. Classic computer vision algorithms are unspecific to the object, so they will pick up any object that is darker than its environment. Therefore, a useful preprocessing step is to exclude some of that noise by applying a mask. With the `create_mask` function, you can include or exclude certain areas of the image from the following analysis steps (`include=True/False`). 

Here, we want to include all of the wite tray that has isopods in it: Hold the left button down and drag the rectangle shaped mask tool over the area you want to include (for more details on the mask and other interaction-tools, see [Tutorial 5](tutorial_5_gui_interactions.ipynb)).

In [4]:
masks = pp.preprocessing.create_mask(image,df_image_data=img_data)

- create mask


### Create size reference

Within the area we masked lies the reference card. We want to include its information in our results files (the pixel-to-mm-ratio), so we supply the image meta DataFrame with `df_image_data=img_data`. However, we do not want the card itself to be detected, so we mask it with the argument `mask=True` and by supplying or mask DataFrame with `df_masks=masks` to have both masks in one place.

<center>
<div style="width:500px; text-align: left">
    
![Adding a scale](_assets/ex1_scale.gif)
    
</div>
</center>

In [5]:
img_data, masks = pp.preprocessing.create_scale(image, 
                                                mask=True,
                                                df_image_data=img_data, 
                                                df_masks=masks)

- measure pixel-to-mm-ratio
Scale set
- add column length
Template selected


Now the `masks` DataFrame contains two masks: the one we created above that *includes* the tray and the one we made to *exclude* the scale reference card:

In [6]:
masks 

,filename,width,height,size_ratio_original,mask,include,coords
0,isopods.jpg,2100,1400,1,mask1,True,"[(249, 180), (1898, 180), (1898, 1353), (249, ..."
0,isopods.jpg,2100,1400,1,scale,False,"[(441, 1017), (682, 1017), (682, 1233), (441, ..."


We can now have a quick look at both masks together by calling the visualization function `show_mask`, followed by `show_image`. For this we should create a new array and *not* overwrite the original image loaded before - here we call it `canvas`:

In [7]:
canvas = pp.visualization.show_masks(image, df_masks=masks, line_width=3)
pp.show_image(canvas)

 - show mask: mask1.
 - show mask: scale.


### Segmentation - 1st attempt

Now that we have removed most of the noise, we can implement an algorithm that segments the imgae into foreground and background (check the [resources section](resources.html#computer-vision) of the documentation. Here we use the `threshold` algorithm that to detect the isopods as foreground and the tray as background. By supplying the mask DataFrame we created before with `df_masks=masks`, we can exclude the unwanted regions of the image: 

In [8]:
image_bin = pp.segmentation.threshold(image, df_masks=masks)

- applying mask: mask1
- applying mask: scale


The resulting array `image_bin` is a binary image where white regions are foreground and black background:

In [9]:
pp.show_image(image_bin)

So far so good, but there is still a lot of noise inside the image (isopod fecal pellets) that we need to remove. Right now, all information about foreground and background is in "pixel-space", i.e. it's a drawn representation of the informative areas. To convert it to coordinate space, and in the later steps extract information from the raw image, we will use `find_contours` on this binary image. But setting the argument `min_area`, we can set a minimum size for the area that the contours should have - 100 pixels should be enough to exclude all fecal pellets. Again, we can supply the image meta DataFrame to concatenate existing information. Afterwards we can draw the contours onto `canvas` with `show_contours` - the detected contours are in green. 

In [10]:
df_contours = pp.segmentation.find_contours(image_bin, df=img_data, min_area=100) 
canvas = pp.visualization.show_contours(canvas, df_contours=df_contours)
pp.show_image(canvas)

### Segmentation - 2nd attempt

Now we have excluded all noise - but some isopods are not well deteced. Mostly the ones with lighter pigmenation, because the contrast they form agains the light background isn't strong enough. We can try a different threshold algorithm by setting the `method` argument: `"adaptive"` algorithms work particularly well with variable contrast levels and lighting. 

Additionally, instead of using the default gray channel (i.e. the average across all three colour channels), we can try to run the thresholding function on a single colour channel. The contrast and signal-to-noise-ratio can be different betweeen the channels. We can select a different channel using the `select_canvas` function - here I isolate all three colour channels, and supply them to `show_image` to show themm all at once, and to evaluate, which colour-channels is suited best: 

In [11]:
r = pp.visualization.select_canvas(image, canvas="r")
g = pp.visualization.select_canvas(image, canvas="g")
b = pp.visualization.select_canvas(image, canvas="b")
pp.show_image([r,g,b], max_dim=500) ## max_dim=reduce window size to 500 pixels on any axis

- red channel
- green channel
- blue channel


Looking at this suggests that the green colour channel will yield the best results: here, even the lighter pigmented isopods have a strong contrast against the tray. We can supply either `g` to `threshold`, or directly select this with the `channel` argument:

In [12]:
image_bin = pp.segmentation.threshold(image, method="adaptive", df_masks=masks, channel="green")
pp.show_image(image_bin)

- applying mask: mask1
- applying mask: scale


### Segmentation - 3rd attempt

Obviously we need to to tune a few things here, waay to many things are being detected. Currently the `"adaptive"` algorithm is on default sensitivity (`blocksize=99`), which we can reduce a bit. Also, we can increase the value for the constant to be subtracted after the tresholding with `constant`. We will try `blocksize=49` and `constant=5`. Afterwards, we plot the detected contours back onto the canvas of the original image, not the green channel image

<center>
<div style="width:800px; text-align: left">

![Binarization](_assets/ex1_binarization.jpg)
        
**Figure 2** - Demonstration of blocksize (19, 99, 199 - left to right) and constant (1, 5 - top and bottom) parameters. Increasing blocksize leads to better structuring of the pixel level information contained in the image (i.e. larger "blocks" of connected pixels can be detected). This is computationally costly, so it will be slow for large images. Also, there is an optimal value beyond which detection performance will decrease. 

</div>
</center>

In [13]:
image_bin = pp.segmentation.threshold(image, method="adaptive", blocksize=49, 
                                      constant=5, df_masks=masks, channel="green")
df_contours = pp.segmentation.find_contours(image_bin, df=img_data, min_area=100)
canvas = pp.visualization.show_contours(image, df_contours=df_contours, line_width=1)
pp.show_image(canvas)

- applying mask: mask1
- applying mask: scale


### Segmentation - final product

That looks pretty good. The last thing we need to take care of are the appendages (we don't want to include those) and the gaps that sometimes form between the segments (we want to have that error to be consistent towards no gaps). Some blurring, and a morphological operation will do the trick (see [the OpenCV docs](https://docs.opencv.org/3.4.9/d9/d61/tutorial_py_morphological_ops.html) and the [resources section](resources.html#computer-vision) of the documentation for more info). Blurring will smooth the contour of the isopods, and a farly large `"cross"` shaped kernel will "cut off" the appendages and other long structures in the binary image. 

In [14]:
pp.show_image(image)

In [15]:
image_blurred = pp.segmentation.blur(image, kernel_size = 15)
image_bin = pp.segmentation.threshold(image_blurred, method="adaptive", blocksize=49, 
                                      constant=5, df_masks=masks,  channel="green")
image_morph = pp.segmentation.morphology(image_bin, operation="open", shape="cross", 
                                         kernel_size = 9, iterations=2)

contours = pp.segmentation.find_contours(image_morph, df=img_data, min_area=250)
canvas = pp.visualization.show_masks(image, df_masks=masks, line_width=3)
canvas = pp.visualization.show_contours(canvas, df_contours=contours, line_width=1)
pp.show_image(canvas)

- applying mask: mask1
- applying mask: scale
 - show mask: mask1.
 - show mask: scale.


### Measuring colour

Ok, this looks good - we now have a DataFrame with the contour-data of all 20 isopods, including their length (`diameter` of the enclosing circle), and the shape (`coords`). Using this information, we can finally extract the colour information from inside the contours. To do so we can supply the original image array along with the contour DataFrame to the `colour_intensity` function:

In [17]:
pigmentation = pp.measurement.colour_intensity(image, df_image_data=img_data, df_contours=contours)

### Export

Done - we can now save the contours and the colour information to csv, as well as the canvas for quality control:

In [18]:
pp.export.save_contours(contours, dirpath=r"../_temp/output")
pp.export.save_colours(pigmentation, dirpath=r"../_temp/output")
pp.export.save_canvas(canvas, dirpath=r"../_temp/output")

- contours saved under ../_temp/output\contours.csv (overwritten).
- colours saved under ../_temp/output\colours.csv (overwritten).
- canvas saved under ../_temp/output\canvas.jpg (overwritten).


## Low throughput

In [ ]:
import phenopype as pp

In [ ]:
filepath = r"images/isopods.jpg"

ct = pp.load_image(filepath, cont=True) ## load image as container

In [ ]:
pp.preprocessing.create_mask(ct)

In [ ]:
pp.preprocessing.create_scale(ct, template=True)

In [ ]:
ct.df_masks

In [ ]:
pp.visualization.show_masks(ct)

In [ ]:
pp.show_image(ct.canvas)

In [ ]:
pp.segmentation.threshold(ct)

In [ ]:
pp.segmentation.find_contours(ct)

In [ ]:
pp.visualization.show_contours(ct)

In [ ]:
pp.show_image(ct.canvas)


In [ ]:
ct.reset()
pp.segmentation.threshold(ct)
pp.segmentation.find_contours(ct, min_area=100)
pp.visualization.show_contours(ct)
pp.visualization.show_masks(ct)
pp.show_image(ct.canvas)

In [ ]:
pp.show_image(ct.image)

## High throughput